In [67]:
import pandas as pd
import requests as r
import json

In [68]:
#setting konfigurasi
with open("../env.json", "r", encoding="utf-8") as file:
    config = json.load(file)  # Membaca dan parsing JSON
varId = 1

## Fungsi Utama

In [69]:
#Get Daftar Data Dinamis BPS Bali
def getVarByDomain(domain='5100'):
    global config
    var_pages=[]
    var = r.get(f'https://webapi.bps.go.id/v1/api/list/model/var/domain/{domain}/key/{config['token']}/')
    pages=var.json()['data'][0]['pages']
    current_page = 1
    while current_page <= pages:
        url = f'https://webapi.bps.go.id/v1/api/list/model/var/domain/{domain}/page/{current_page}/key/{config['token']}/'
        var_page=r.get(url)
        var_pages.append(pd.json_normalize(var_page.json()['data'][1]))
        current_page=current_page+1
    var_pages_df = pd.concat(var_pages)
    return var_pages_df

In [70]:
#Get Daftar Periode
def getPriodeByVar(domain='5100',varId=varId):
    global config
    current_page = 1
    url = f'https://webapi.bps.go.id/v1/api/list/model/th/domain/{domain}/var/{varId}/page/{current_page}/key/{config['token']}/'
    dt_json = r.get(url)
    pages = dt_json.json()['data'][0]['pages']
    dt_all = []
    while current_page <= pages:
        url = f'https://webapi.bps.go.id/v1/api/list/model/th/domain/{domain}/var/{varId}/page/{current_page}/key/{config['token']}/'
        dt = pd.json_normalize(dt_json.json()['data'][1])
        dt_all.append(dt)
        current_page = current_page+1
    dt_all = pd.concat(dt_all)
    dt_all['varId']=varId
    return dt_all

In [71]:
def crossJoin(left,right):
    if left.empty:
        left = right
    else:
        left = pd.merge(left,right,how='cross')
    return left
def getDataByVarId(varId=varId,domain='5100'):
    global config
    #Banyaknya Wisatawan Mancanegara Bulanan ke Bali Menurut Pintu Masuk
    dt_th = getPriodeByVar(domain=domain,varId=varId)
    dt_all = []
    for i in dt_th.index:
        th_id = dt_th.iloc[i]['th_id']
        url = f'https://webapi.bps.go.id/v1/api/list/model/data/domain/{domain}/var/{varId}/key/{config['token']}/th/{th_id}/'
        dt_json = r.get(url)
        attr = pd.DataFrame()
        if 'vervar' in dt_json.json().keys():
            vervar=pd.json_normalize(dt_json.json()['vervar'])
            vervar=vervar.rename(columns={'val': 'verval_val','label': 'vervar_label'})
            attr = crossJoin(attr,vervar)
        if 'var' in dt_json.json().keys():
            var=pd.json_normalize(dt_json.json()['var'])
            var=var.rename(columns={'val': 'var_val','label': 'var_label'})
            var=var[['var_val']]
            attr = crossJoin(attr,var)
        if 'turvar' in dt_json.json().keys():
            turvar=pd.json_normalize(dt_json.json()['turvar'])
            turvar=turvar.rename(columns={'val': 'turvar_val','label': 'turvar_label'})
            attr = crossJoin(attr,turvar)
        if 'tahun' in dt_json.json().keys():
            tahun=pd.json_normalize(dt_json.json()['tahun'])
            tahun=tahun.rename(columns={'val': 'tahun_val','label': 'tahun_label'})
            attr = crossJoin(attr,tahun)
        if 'turtahun' in dt_json.json().keys():
            turtahun=pd.json_normalize(dt_json.json()['turtahun'])
            turtahun=turtahun.rename(columns={'val': 'turtahun_val','label': 'turtahun_label'})
            attr = crossJoin(attr,turtahun)
        val_columns = [col for col in attr.columns if col.endswith('val')]
        attr['id']=attr[val_columns].astype('str').agg(''.join, axis=1)
        dt_content=pd.json_normalize(dt_json.json()['datacontent']).transpose().reset_index().rename(columns={'index':'content_val',0:'content_label'})
        attr = pd.merge(attr,dt_content,left_on='id',right_on='content_val',how='left')
        attr = attr[attr.content_label.notna()]
        dt_all.append(attr)
    dt_all=pd.concat(dt_all)
    return dt_all

In [79]:
def clean_tag_html(data):
    data["vervar_label"] = data["vervar_label"].str.replace(r"<.*?>", "", regex=True)
    return data
    

In [72]:
daftar_data=getVarByDomain(domain='5100')

### (1) Inflasi Tahunan Kota Denpasar, Singaraja, dan Nasional

In [82]:
data1=getDataByVarId(domain='5100',varId=varId)
data1.head()

,verval_val,vervar_label,var_val,turvar_val,turvar_label,tahun_val,tahun_label,turtahun_val,turtahun_label,id,content_val,content_label
0,1,Bahan Makanan,47,0,Tidak ada,119,2019,0,Tahun,14701190,14701190,1.97
1,2,"Makanan Jadi, Minuman, Rokok, dan Tembakau",47,0,Tidak ada,119,2019,0,Tahun,24701190,24701190,3.82
2,3,Perumahan,47,0,Tidak ada,119,2019,0,Tahun,34701190,34701190,0.60
3,4,Sandang,47,0,Tidak ada,119,2019,0,Tahun,44701190,44701190,4.02
4,5,Kesehatan,47,0,Tidak ada,119,2019,0,Tahun,54701190,54701190,5.67


In [83]:
#proses cleaning
data1=data1[['verval_val','vervar_label','tahun_label','content_label']]
#proses rename colomn
data1_tb=data1.rename(columns={'vervar_label' : 'wilayah_inflasi','tahun_label' : 'tahun','content_label' : 'inflasi'})
#pivoting tabel
data1_tb=data1_tb.pivot_table(index=['verval_val','wilayah_inflasi'],columns=['tahun'],values=['inflasi'])
#multi index jadikan 1
data1_tb.columns = ['_'.join(col).strip() if isinstance(col, tuple) else col
                   for col in data1_tb.columns.values]
data1_tb=data1_tb.reset_index()
data1_tb.drop(columns=['verval_val'])
data1_tb.head()

,verval_val,wilayah_inflasi,inflasi_2014,inflasi_2015,inflasi_2016,inflasi_2017,inflasi_2018,inflasi_2019
0,1,Bahan Makanan,8.09,4.74,9.98,-3.39,4.33,1.97
1,2,"Makanan Jadi, Minuman, Rokok, dan Tembakau",15.00,4.22,4.08,3.15,0.77,3.82
2,3,Perumahan,9.97,3.33,1.83,8.44,-3.07,0.60
3,4,Sandang,6.25,7.73,8.47,6.53,4.42,4.02
4,5,Kesehatan,1.45,2.61,4.65,5.02,2.67,5.67


In [75]:
# Export to Excel
data1_tb.to_excel(f'output/data_{varId}.xlsx', index=False)

### (47) Inflasi Tahunan Kota Singaraja Menurut 7 Kelompok Pengeluaran

In [84]:
varId = 47
data47=getDataByVarId(domain='5100',varId=varId)
data47.head()

,verval_val,vervar_label,var_val,turvar_val,turvar_label,tahun_val,tahun_label,turtahun_val,turtahun_label,id,content_val,content_label
0,1,Bahan Makanan,47,0,Tidak ada,119,2019,0,Tahun,14701190,14701190,1.97
1,2,"Makanan Jadi, Minuman, Rokok, dan Tembakau",47,0,Tidak ada,119,2019,0,Tahun,24701190,24701190,3.82
2,3,Perumahan,47,0,Tidak ada,119,2019,0,Tahun,34701190,34701190,0.60
3,4,Sandang,47,0,Tidak ada,119,2019,0,Tahun,44701190,44701190,4.02
4,5,Kesehatan,47,0,Tidak ada,119,2019,0,Tahun,54701190,54701190,5.67


In [85]:
#proses cleaning
data47=data47[['verval_val','vervar_label','tahun_label','content_label']]
data47 = clean_tag_html(data47)
#proses rename colomn
data47_tb=data47.rename(columns={'vervar_label' : 'kelompok_inflasi','tahun_label' : 'tahun','content_label' : 'inflasi'})
#pivoting tabel
data47_tb=data47_tb.pivot_table(index=['verval_val','kelompok_inflasi'],columns=['tahun'],values=['inflasi'])
#multi index jadikan 1
data47_tb.columns = ['_'.join(col).strip() if isinstance(col, tuple) else col
                   for col in data47_tb.columns.values]
data47_tb=data47_tb.reset_index()
data47_tb.drop(columns=['verval_val'])
data47_tb.head()

,verval_val,kelompok_inflasi,inflasi_2014,inflasi_2015,inflasi_2016,inflasi_2017,inflasi_2018,inflasi_2019
0,1,Bahan Makanan,8.09,4.74,9.98,-3.39,4.33,1.97
1,2,"Makanan Jadi, Minuman, Rokok, dan Tembakau",15.00,4.22,4.08,3.15,0.77,3.82
2,3,Perumahan,9.97,3.33,1.83,8.44,-3.07,0.60
3,4,Sandang,6.25,7.73,8.47,6.53,4.42,4.02
4,5,Kesehatan,1.45,2.61,4.65,5.02,2.67,5.67


In [86]:
# Export to Excel
data47_tb.to_excel(f'output/data_{varId}.xlsx', index=False)